In [ ]:
%matplotlib inline

# Preliminaries 

In [ ]:
import os
import pandas as pd
from ase.io import read
import torch
import time
import numpy as np
from sklearn import preprocessing, linear_model, pipeline, model_selection
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [2]:
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist

In [3]:
from kymatio.torch import HarmonicScattering3D

In [4]:
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

In [5]:
from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians

In [6]:
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

# Extract features

In [7]:
# === Chargement des positions et charges ===
def extract_features(xyz_path):
    atoms = read(xyz_path)
    return atoms.get_positions(), atoms.get_atomic_numbers()

def load_all_xyz(folder_path, max_atoms=23):
    positions_list, charges_list = [], []
    xyz_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.xyz')])
    for f in xyz_files:
        pos, chg = extract_features(os.path.join(folder_path, f))
        pos_padded = np.zeros((max_atoms, 3))
        chg_padded = np.zeros((max_atoms,))
        n_atoms = pos.shape[0]
        pos_padded[:n_atoms] = pos
        chg_padded[:n_atoms] = chg
        positions_list.append(pos_padded)
        charges_list.append(chg_padded)
    return np.stack(positions_list), np.stack(charges_list)

# === Chargement des énergies ===
data_dir = "./"
xyz_dir = os.path.join(data_dir, "atoms", "train")
csv_path = os.path.join(data_dir, "energies", "train.csv")
energies_df = pd.read_csv(csv_path)
energies_df["id"] = energies_df["id"].astype(str)
energies_df = energies_df.sort_values("id").to_numpy()

In [8]:
# === Chargement des positions ===
pos_train, full_charges_train = load_all_xyz(xyz_dir, max_atoms=23)
n_molecules = pos_train.shape[0]

# === Calcul des charges de valence ===
valence_charges = np.zeros_like(full_charges_train)
valence_charges += (full_charges_train <= 2) * full_charges_train
valence_charges += np.logical_and(full_charges_train > 2, full_charges_train <= 10) * (full_charges_train - 2)
valence_charges += np.logical_and(full_charges_train > 10, full_charges_train <= 18) * (full_charges_train - 10)

# === Normalisation des positions ===
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf
for i in range(n_molecules):
    n_atoms = np.sum(full_charges_train[i] != 0)
    min_dist = min(min_dist, pdist(pos_train[i, :n_atoms]).min())
delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos_train *= delta / min_dist

In [9]:
# === Paramètres scattering ===
M, N, O = 96, 64, 48
grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)
J = 4
L = 3
integral_powers = [0.5, 1.0, 2.0, 3.0, 4.0]
scattering = HarmonicScattering3D(J=J, shape=(M, N, O), L=L, sigma_0=sigma, integral_powers=integral_powers)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scattering.to(device)
print("Using", device)

Using cuda


In [11]:
# === Boucle de batching ===
order_0, orders_1_and_2 = [], []
batch_size = 8
n_batches = int(np.ceil(n_molecules / batch_size))

for i in tqdm(range(n_batches)):
    start, end = i * batch_size, min((i + 1) * batch_size, n_molecules)
    pos_batch = pos_train[start:end]
    full_batch = full_charges_train[start:end]
    val_batch = valence_charges[start:end]

    full_density = generate_weighted_sum_of_gaussians(grid, pos_batch, full_batch, sigma)
    val_density = generate_weighted_sum_of_gaussians(grid, pos_batch, val_batch, sigma)
    full_density = torch.from_numpy(full_density).to(device).float()
    val_density = torch.from_numpy(val_density).to(device).float()
    core_density = full_density - val_density
    bond_density = torch.abs(val_density - core_density)

    # Order 0
    f0 = TorchBackend3D.compute_integrals(full_density, integral_powers)
    v0 = TorchBackend3D.compute_integrals(val_density, integral_powers)
    c0 = TorchBackend3D.compute_integrals(core_density, integral_powers)
    b0 = TorchBackend3D.compute_integrals(bond_density, integral_powers)

    # Orders 1 & 2
    fs = scattering(full_density)
    vs = scattering(val_density)
    cs = scattering(core_density)
    bs = scattering(bond_density)

    # Stack
    order_0.append(torch.stack([f0, v0, c0, b0], dim=-1))
    orders_1_and_2.append(torch.stack([fs, vs, cs, bs], dim=-1))

# === Feature flattening ===
order_0_train = torch.cat(order_0, dim=0).cpu().numpy().reshape((n_molecules, -1))
orders_1_and_2_train = torch.cat(orders_1_and_2, dim=0).cpu().numpy().reshape((n_molecules, -1))

100%|██████████| 824/824 [1:24:40<00:00,  6.17s/it]


In [12]:
basename = 'molecule_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

cache_dir = get_cache_dir("results/train/")

filename = os.path.join(cache_dir, 'order_0_' + basename)
np.save(filename, order_0_train)

filename = os.path.join(cache_dir, 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2_train)

# Test

In [8]:
pos_test, full_charges_test = load_all_xyz("./atoms/test/", max_atoms=23)
n_molecules_test = pos_test.shape[0]

# === Calcul des charges de valence ===
valence_charges = np.zeros_like(full_charges_test)
valence_charges += (full_charges_test <= 2) * full_charges_test
valence_charges += np.logical_and(full_charges_test > 2, full_charges_test <= 10) * (full_charges_test - 2)
valence_charges += np.logical_and(full_charges_test > 10, full_charges_test <= 18) * (full_charges_test - 10)

# === Normalisation des positions ===
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf
for i in range(n_molecules_test):
    n_atoms = np.sum(full_charges_test[i] != 0)
    min_dist = min(min_dist, pdist(pos_test[i, :n_atoms]).min())
delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos_test *= delta / min_dist

In [9]:
# === Paramètres scattering ===
M, N, O = 96, 64, 48
grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)
J = 4
L = 3
integral_powers = [0.5, 1.0, 2.0, 3.0, 4.0]
scattering = HarmonicScattering3D(J=J, shape=(M, N, O), L=L, sigma_0=sigma, integral_powers=integral_powers)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scattering.to(device)
print("Using", device)

Using cuda


In [11]:
# === Boucle de batching ===
order_0, orders_1_and_2 = [], []
batch_size = 8
n_batches = int(np.ceil(n_molecules_test / batch_size))

for i in tqdm(range(n_batches)):
    start, end = i * batch_size, min((i + 1) * batch_size, n_molecules_test)
    pos_batch = pos_test[start:end]
    full_batch = full_charges_test[start:end]
    val_batch = valence_charges[start:end]

    full_density = generate_weighted_sum_of_gaussians(grid, pos_batch, full_batch, sigma)
    val_density = generate_weighted_sum_of_gaussians(grid, pos_batch, val_batch, sigma)
    full_density = torch.from_numpy(full_density).to(device).float()
    val_density = torch.from_numpy(val_density).to(device).float()
    core_density = full_density - val_density
    bond_density = torch.abs(val_density - core_density)

    # Order 0
    f0 = TorchBackend3D.compute_integrals(full_density, integral_powers)
    v0 = TorchBackend3D.compute_integrals(val_density, integral_powers)
    c0 = TorchBackend3D.compute_integrals(core_density, integral_powers)
    b0 = TorchBackend3D.compute_integrals(bond_density, integral_powers)

    # Orders 1 & 2
    fs = scattering(full_density)
    vs = scattering(val_density)
    cs = scattering(core_density)
    bs = scattering(bond_density)

    # Stack
    order_0.append(torch.stack([f0, v0, c0, b0], dim=-1))
    orders_1_and_2.append(torch.stack([fs, vs, cs, bs], dim=-1))

# === Feature flattening ===
order_0_test = torch.cat(order_0, dim=0).cpu().numpy().reshape((n_molecules_test, -1))
orders_1_and_2_test = torch.cat(orders_1_and_2, dim=0).cpu().numpy().reshape((n_molecules_test, -1))

100%|██████████| 206/206 [26:34<00:00,  7.74s/it]


In [13]:
basename = 'molecule_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

cache_dir = get_cache_dir("results/test/")

filename = os.path.join(cache_dir, 'order_0_' + basename)
np.save(filename, order_0_test)

filename = os.path.join(cache_dir, 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2_test)